In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# =========================
# 0) CONFIG
# =========================
# Metti qui i tuoi file o folder (supporta anche wildcard)
DISTIL_PATH = r"SECUREBERT/1-CTI_DATASET.csv"
SECURE_PATH = r"SECUREBERT/SECURE_DBS/1-CTI_DATASET.csv"

# Chiave per matchare lo stesso messaggio tra dataset
# Consigliato: (group_id, msg_id) o (chat_name, msg_id)
KEY_COLS = ["group_id", "msg_id"]

# Soglia (se vuoi controllare / ricalcolare)
THRESH = 0.60

# =========================
# 1) LOAD (singolo file o wildcard)
# =========================
def load_many(path_pattern: str) -> pd.DataFrame:
    paths = sorted(Path().glob(path_pattern))
    if not paths:
        raise FileNotFoundError(f"Nessun file trovato per pattern: {path_pattern}")
    dfs = [pd.read_csv(p) for p in paths]
    df = pd.concat(dfs, ignore_index=True)
    return df

secure = load_many(SECURE_PATH)
distil = load_many(DISTIL_PATH)

# Normalizza tipi e colonne chiave
for df, name in [(secure, "secure"), (distil, "distil")]:
    missing = [c for c in KEY_COLS if c not in df.columns]
    if missing:
        raise ValueError(f"Mancano colonne chiave {missing} in dataset {name}")
    # msg_id spesso è int, ma per sicurezza lo forziamo a stringa uniforme
    df["msg_id"] = df["msg_id"].astype(str)
    df["group_id"] = df["group_id"].astype(str)

# (Opzionale) se vuoi verificare che i CSV rispettino già la soglia:
if "cyber_score" in secure.columns:
    print("Secure: % sotto soglia (dovrebbe essere ~0 se hai già filtrato):",
          (secure["cyber_score"] < THRESH).mean())
if "cyber_score" in distil.columns:
    print("Distil: % sotto soglia (dovrebbe essere ~0 se hai già filtrato):",
          (distil["cyber_score"] < THRESH).mean())

# Dedup: nel caso in cui lo stesso msg sia comparso più volte
secure = secure.drop_duplicates(subset=KEY_COLS).copy()
distil = distil.drop_duplicates(subset=KEY_COLS).copy()

# =========================
# 2) SET METRICS (overlap)
# =========================
secure_keys = set(map(tuple, secure[KEY_COLS].to_numpy()))
distil_keys = set(map(tuple, distil[KEY_COLS].to_numpy()))

inter = secure_keys & distil_keys
union = secure_keys | distil_keys

nS = len(secure_keys)
nD = len(distil_keys)
nI = len(inter)
nU = len(union)

pct_D_in_S = (nI / nD) * 100 if nD else 0
pct_S_in_D = (nI / nS) * 100 if nS else 0
jaccard = (nI / nU) if nU else 0

print("\n=== OVERLAP METRICS ===")
print(f"Secure retained: {nS:,}")
print(f"Distil retained: {nD:,}")
print(f"Intersection:    {nI:,}")
print(f"Union:           {nU:,}")
print(f"% Distil covered by Secure (|D∩S|/|D|): {pct_D_in_S:.2f}%")
print(f"% Secure covered by Distil (|D∩S|/|S|): {pct_S_in_D:.2f}%")
print(f"Jaccard (|D∩S|/|D∪S|): {jaccard:.4f}")

# =========================
# 3) BREAKDOWN PER GROUP
# =========================
# Conteggio per gruppo
Sg = secure.groupby("chat_name", dropna=False).size().rename("secure_n").reset_index()
Dg = distil.groupby("chat_name", dropna=False).size().rename("distil_n").reset_index()

# Intersection per gruppo: facciamo merge sulle chiavi e contiamo
merged = secure[KEY_COLS + ["chat_name"]].merge(
    distil[KEY_COLS + ["chat_name"]],
    on=KEY_COLS,
    how="inner",
    suffixes=("_S", "_D"),
)
Ig = merged.groupby("chat_name_S", dropna=False).size().rename("intersection_n").reset_index()
Ig = Ig.rename(columns={"chat_name_S": "chat_name"})

by_group = Sg.merge(Dg, on="chat_name", how="outer").merge(Ig, on="chat_name", how="left")
by_group = by_group.fillna(0)
by_group["secure_n"] = by_group["secure_n"].astype(int)
by_group["distil_n"] = by_group["distil_n"].astype(int)
by_group["intersection_n"] = by_group["intersection_n"].astype(int)

# percentuali per gruppo
by_group["pct_D_in_S"] = np.where(by_group["distil_n"] > 0,
                                  100 * by_group["intersection_n"] / by_group["distil_n"],
                                  0)
by_group["pct_S_in_D"] = np.where(by_group["secure_n"] > 0,
                                  100 * by_group["intersection_n"] / by_group["secure_n"],
                                  0)

# ordina per divergenza: (secure - distil) o solo secure
by_group["delta_S_minus_D"] = by_group["secure_n"] - by_group["distil_n"]

print("\n=== TOP GROUPS BY (Secure - Distil) ===")
print(by_group.sort_values("delta_S_minus_D", ascending=False)
      .head(10)[["chat_name", "secure_n", "distil_n", "intersection_n", "delta_S_minus_D", "pct_D_in_S", "pct_S_in_D"]])

# =========================
# 4) DISCORDANT SETS: only-secure / only-distil
# =========================
only_secure_keys = secure_keys - distil_keys
only_distil_keys = distil_keys - secure_keys

only_secure = secure.merge(
    pd.DataFrame(list(only_secure_keys), columns=KEY_COLS),
    on=KEY_COLS,
    how="inner"
)
only_distil = distil.merge(
    pd.DataFrame(list(only_distil_keys), columns=KEY_COLS),
    on=KEY_COLS,
    how="inner"
)

print("\n=== DISCORDANT COUNTS ===")
print(f"Only Secure: {len(only_secure):,}")
print(f"Only Distil: {len(only_distil):,}")

# =========================
# 5) SIMPLE TEXT STATS + PATTERN COUNTS (discordant analysis)
# =========================
def add_text_stats(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["text_len"] = df["text"].astype(str).str.len()
    df["word_len"] = df["text"].astype(str).str.split().apply(len)

    # pattern "grezzi" (non IoC extraction completa, solo indicatori veloci)
    df["has_url"] = df["text"].astype(str).str.contains(r"(https?://|www\.)", regex=True)
    df["has_cve"] = df["text"].astype(str).str.contains(r"\bCVE-\d{4}-\d+\b", regex=True, case=False)
    df["has_ip"]  = df["text"].astype(str).str.contains(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", regex=True)
    df["has_domain_like"] = df["text"].astype(str).str.contains(
        r"\b(?:[a-zA-Z0-9-]+\.)+(?:com|org|net|io|ru|cn|it|uk|gov)\b",
        regex=True
    )
    return df

only_secure = add_text_stats(only_secure)
only_distil = add_text_stats(only_distil)

def summarize(df: pd.DataFrame, name: str):
    if df.empty:
        print(f"\n[{name}] dataset vuoto.")
        return
    print(f"\n=== {name} SUMMARY ===")
    print(f"N: {len(df):,}")
    print("text_len (mean/median):", df["text_len"].mean().round(2), "/", df["text_len"].median())
    print("word_len (mean/median):", df["word_len"].mean().round(2), "/", df["word_len"].median())
    for col in ["has_url", "has_cve", "has_ip", "has_domain_like"]:
        print(f"{col}: {(df[col].mean()*100):.2f}%")

summarize(only_secure, "ONLY SECURE")
summarize(only_distil, "ONLY DISTIL")

# =========================
# 6) OPTIONAL: SAVE REPORT CSVs
# =========================
out_dir = Path("compare_reports")
out_dir.mkdir(exist_ok=True)

by_group.sort_values("delta_S_minus_D", ascending=False).to_csv(out_dir / "by_group_overlap.csv", index=False)
only_secure.to_csv(out_dir / "only_secure.csv", index=False)
only_distil.to_csv(out_dir / "only_distil.csv", index=False)

print(f"\nSaved reports to: {out_dir.resolve()}")


Secure: % sotto soglia (dovrebbe essere ~0 se hai già filtrato): 0.0
Distil: % sotto soglia (dovrebbe essere ~0 se hai già filtrato): 0.0

=== OVERLAP METRICS ===
Secure retained: 367,457
Distil retained: 102,654
Intersection:    85,099
Union:           385,012
% Distil covered by Secure (|D∩S|/|D|): 82.90%
% Secure covered by Distil (|D∩S|/|S|): 23.16%
Jaccard (|D∩S|/|D∪S|): 0.2210

=== TOP GROUPS BY (Secure - Distil) ===
                                            chat_name  secure_n  distil_n  \
7                                           Only Dark    155579     49962   
9                                 Теневой Даркнет Чат     70883     14518   
11                                       Чат доксеров     63242      8548   
1                                            DDOS交流攻击     41765     12509   
10                          Хакеры |Чат| 𝓗𝓪𝓬𝓴𝓮𝓻𝓼 𝓬𝓱𝓪𝓽     16344      1654   
6                                   Mikrotik-Training      8831      1814   
8                                 

C:\Users\apisp\AppData\Local\Temp\ipykernel_21692\423543095.py:149: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_url"] = df["text"].astype(str).str.contains(r"(https?://|www\.)", regex=True)
C:\Users\apisp\AppData\Local\Temp\ipykernel_21692\423543095.py:149: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df["has_url"] = df["text"].astype(str).str.contains(r"(https?://|www\.)", regex=True)



=== ONLY SECURE SUMMARY ===
N: 282,358
text_len (mean/median): 213.38 / 146.0
word_len (mean/median): 28.25 / 21.0
has_url: 1.70%
has_cve: 0.00%
has_ip: 0.06%
has_domain_like: 0.11%

=== ONLY DISTIL SUMMARY ===
N: 17,555
text_len (mean/median): 156.17 / 41.0
word_len (mean/median): 21.93 / 6.0
has_url: 2.09%
has_cve: 0.00%
has_ip: 2.49%
has_domain_like: 0.85%

Saved reports to: C:\Users\apisp\OneDrive\Desktop\PoliTo\II Anno\DPA\IoC_on_Telegram\compare_reports
